In [ ]:
#!/usr/bin/env python3
!pip install rasterio -q
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import rasterio
import numpy as np
from collections import defaultdict

from google.colab import drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 61.2 MB/s eta 0:00:00
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# ----------------------
# 1. Load all predictor data
# ----------------------

continents = {
    "Africa": "hybas_af",
    "Asia": "hybas_as",
    "SouthAmerica": "hybas_sa",
    "NorthAmerica": "hybas_na",
    "Europe": "hybas_eu",
    "Oceanie": "hybas_au",
    "Greenland": "hybas_gr",
    "Siberia": "hybas_si",
    "Arctic": "hybas_ar"
}

base_path = "/content/drive/MyDrive/data_scriptie/predictor_data"
continent_shapefiles = [
    f"{base_path}/{continent}/{basename}_lev{str(9).zfill(2)}_v1c.shp"
    for continent, basename in continents.items()
]

gdfs = [gpd.read_file(path) for path in continent_shapefiles]
hydrobasins = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), crs=gdfs[0].crs)

print(f"Loaded {len(hydrobasins)} HydroBASINS catchments for level {level}.")


Loaded 508190 HydroBASINS catchments for level 9.


In [ ]:
# ----------------------
# 2. Load HydroATLAS attributes and merge on HYBAS_ID
# ----------------------
atlas_file = "/content/drive/MyDrive/data_scriptie/predictor_data/BasinAtlas/ShapeFiles/BasinATLAS_v10_lev09.shp"
atlas = gpd.read_file(atlas_file)
common_cols = set(hydrobasins.columns).intersection(atlas.columns)
atlas_extra = atlas[[c for c in atlas.columns if c not in common_cols or c=='HYBAS_ID']]
gdf_basins = hydrobasins.merge(
    atlas_extra.drop(columns='geometry', errors='ignore'), on='HYBAS_ID', how='left'
)

print("Merged HydroBASINS with HydroATLAS attributes.")

Merged HydroBASINS with HydroATLAS attributes.


In [ ]:
# ----------------------
# 3. Compute centroids in projected CRS then back to original
# ----------------------
projected_crs = "EPSG:3857"
gdf_proj = gdf_basins.to_crs(projected_crs)
gdf_proj['geometry'] = gdf_proj.geometry.centroid
gdf_centroids = gdf_proj.to_crs(gdf_basins.crs)


In [ ]:
# ----------------------
# 4. Sample raster predictors on basin centroids
# ----------------------
predictor_paths = {
    'flow_direction':    "/content/drive/MyDrive/data_scriptie/hyd_glo_dir_15s.tif",
    'flow_length':       "/content/drive/MyDrive/data_scriptie/hyd_glo_lup_15s.tif",
    'flow_accumulation': "/content/drive/MyDrive/data_scriptie/hyd_glo_acc_15s.tif",
}

def sample_all_predictors(gdf, predictor_paths):
    out = gdf.copy()
    for name, path in predictor_paths.items():
        with rasterio.open(path) as src:
            pts = out.to_crs(src.crs) if out.crs != src.crs else out
            coords = [(pt.x, pt.y) for pt in pts.geometry]
            vals = np.array([v[0] for v in src.sample(coords)])
            out[name] = np.where(vals == src.nodata, np.nan, vals)
    return out

gdf_preds = sample_all_predictors(gdf_centroids, predictor_paths)
print("Sampled raster predictors.")

Sampled raster predictors.


In [ ]:
# ----------------------
# 5. Build network maps and get_upstream_ids
# ----------------------
net_cols = ['NEXT_DOWN', 'NEXT_SINK', 'MAIN_BAS', 'UP_AREA', 'DIST_SINK', 'DIST_MAIN']
net_df = gdf_basins.set_index('HYBAS_ID')[net_cols]
down_map = net_df['NEXT_DOWN'].dropna().astype(int).to_dict()
up_map = defaultdict(list)
for src_id, dst_id in down_map.items():
    up_map[dst_id].append(src_id)

def get_upstream_ids(basin_id):
    visited = set()
    stack = [basin_id]
    while stack:
        curr = stack.pop()
        for up in up_map.get(curr, []):
            if up not in visited:
                visited.add(up)
                stack.append(up)
    return visited

In [ ]:
# ----------------------
# 6. Compute upstream-derived features
# ----------------------

gdf_preds['upstream_count'] = gdf_preds['HYBAS_ID'].apply(lambda h: len(get_upstream_ids(h)))
for name in predictor_paths:
    col_local = name
    col_up = f'upstream_mean_{name}'
    means = []
    temp = gdf_preds.set_index('HYBAS_ID')
    for h in gdf_preds['HYBAS_ID']:
        ups = get_upstream_ids(h)
        means.append(temp.loc[list(ups), col_local].mean() if ups else np.nan)
    gdf_preds[col_up] = means
print("Computed upstream means.")

Computed upstream means.


In [ ]:
# ----------------------
# 7. Clean features: drop IDs, suffixes
# ----------------------
df = gdf_preds.drop(columns=['geometry'], errors='ignore')
id_prefixes = ['NEXT_DOWN','NEXT_SINK','MAIN_BAS','PFAF_ID','SORT','ORDER']
drop_id = [c for c in df.columns if any(c.startswith(pref) for pref in id_prefixes)]
drop_suf = [c for c in df.columns if c.endswith(('_x','_y'))]
df.drop(columns=drop_id + drop_suf, inplace=True, errors='ignore')


for col in ['UP_AREA','SUB_AREA','DIST_MAIN','flow_length','flow_accumulation']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

df.rename(columns={'UP_AREA':'up_area','SUB_AREA':'sub_area','DIST_MAIN':'dist_main'}, inplace=True)

In [ ]:
# ----------------------
# 8. Apply log1p transforms
# ----------------------
# Print raw values
print(df[['up_area','sub_area','dist_main']].head(10))
# Compute logs
df['up_area_log']   = np.log1p(df['up_area'])
df['sub_area_log']  = np.log1p(df['sub_area'])
df['dist_main_log'] = np.log1p(df['dist_main'])

   up_area  sub_area  dist_main
0     11.0      11.0        0.0
1    416.9     416.9        0.0
2    186.9     186.8        0.0
3    235.6     235.6        0.0
4      8.3       8.3        0.0
5    328.6     328.6        0.0
6      2.6       2.6        0.0
7    686.3     686.2        0.0
8     14.9      14.9        0.0
9   2925.8      17.1        0.0


In [ ]:
# ----------------------
# 9. Save final feature matrix
# ----------------------
up_cols = [c for c in df.columns if c.startswith("upstream_mean_")]
for col in up_cols:
    df[col] = df[col].fillna(0)

for col in ['sub_area','up_area', 'dist_main', 'flow_length', 'flow_accumulation']:
    if col in df.columns:
        df.drop(columns=[col], inplace=True)

df = pd.get_dummies(df, columns=['flow_direction'], prefix='dir', dtype=int)

atlas_extra = atlas[['HYBAS_ID', 'geometry']]
gdf_feats = pd.merge(df, atlas_extra, on='HYBAS_ID')

out_file = '/content/drive/MyDrive/data_scriptie/Output/features_final.csv'

gdf_feats.to_csv(
    out_file,
    index=False
)
print(f"Saved features to {out_file}.")

Saved features to /content/drive/MyDrive/data_scriptie/Output/features_final.csv.
